# Sequential

The next step after calling a language model is to make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another.

The recommended way to do this is using the LangChain Expression Language. The legacy way is using the `SequentialChain`, which we continue to document here for backwards compatibility.

As a toy example, let's suppose we want to create a chain that first creates a play synopsis and then generates a play review based on the synopsis.

In [1]:
from langchain.prompts import PromptTemplate

synopsis_prompt = PromptTemplate.from_template(
    """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
)

review_prompt = PromptTemplate.from_template(
    """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
)

## Using LCEL

Creating a sequence of calls (to LLMs or any other component/arbitrary function) is precisely what LangChain Expression Language was designed for.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

llm = ChatOpenAI()
chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
chain.invoke({"title": "Tragedy at sunset on the beach"})

'In "Tragedy at Sunset on the Beach," playwright has crafted a deeply affecting drama that delves into the complexities of human relationships and the consequences that arise from one fateful evening. Set against the breathtaking backdrop of a serene beach at sunset, the play takes audiences on an emotional journey as it explores the lives of four individuals whose paths intertwine in unexpected and tragic ways.\n\nAt the center of the story is Sarah, a young woman grappling with the recent loss of her husband. Seeking solace and a fresh start, she embarks on a solitary trip to the beach, hoping to find peace and clarity. It is here that she encounters James, a charismatic but troubled artist, lost in his own world of anguish and self-doubt. The unlikely connection they form becomes the catalyst for a series of heart-wrenching events, as their emotional baggage and personal demons collide.\n\nThe play skillfully weaves together the narratives of Sarah, James, and Rachel, Sarah\'s best 

If we wanted to get back the synopsis as well we could do:

In [7]:
from langchain.schema.runnable import RunnablePassthrough

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(review=review_chain)
chain.invoke({"title": "Tragedy at sunset on the beach"})

{'synopsis': 'Tragedy at Sunset on the Beach is a gripping and emotionally charged drama that delves into the complexities of human relationships and the fragility of life. Set against the backdrop of a picturesque beach at sunset, the play follows a group of friends who gather to celebrate a joyous occasion.\n\nAs the sun begins its descent, tensions simmer beneath the surface, and long-held secrets and resentments come to light. The characters find themselves entangled in a web of love, betrayal, and loss, as they confront their deepest fears and desires.\n\nThe main focus revolves around Sarah, a vibrant and free-spirited woman who becomes the center of a tragic event. Through a series of flashback scenes, we witness the unraveling of her life, exploring her complicated relationships with her closest friends and romantic partners.\n\nThe play explores themes of regret, redemption, and the consequences of our choices. It delves into the human condition, questioning the nature of happ

Head to the [LCEL](/docs/expression_language) section for more on the interface, built-in features, and cookbook examples.

## [Legacy] SequentialChain

:::note This is a legacy class, using LCEL as shown above is preffered.

Sequential chains allow you to connect multiple chains and compose them into pipelines that execute some specific scenario. There are two types of sequential chains:

- `SimpleSequentialChain`: The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.
- `SequentialChain`: A more general form of sequential chains, allowing for multiple inputs/outputs.

### SimpleSequentialChain

In [8]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=0.7)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt)

In [9]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=0.7)
review_chain = LLMChain(llm=llm, prompt=review_prompt)

In [10]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...


Tragedy at Sunset on the Beach is a modern tragedy about a young couple in love. The couple, Jack and Jill, are deeply in love and plan to spend the day together on the beach at sunset. However, when they arrive, they are shocked to discover that the beach is an abandoned, dilapidated wasteland. With no one else around, they explore the beach and start to reminisce about their relationship and the good times they’ve shared. 

But then, out of the blue, a mysterious figure emerges from the shadows and reveals a dark secret. The figure tells the couple that the beach is no ordinary beach, but is in fact the site of a terrible tragedy that took place many years ago. As the figure explains what happened, Jack and Jill become overwhelmed with grief. 

In the end, Jack and Jill are forced to confront the truth about the tragedy and its consequences. The play is ultimately a reflection on the power of tragedy and the human capacity to confront

In [11]:
print(review)



Tragedy at Sunset on the Beach is a powerful, thought-provoking modern tragedy that is sure to leave a lasting impression on its audience. The play follows the story of Jack and Jill, a young couple deeply in love, as they explore an abandoned beach and discover a dark secret from the past.

The play brilliantly captures the raw emotions of Jack and Jill as they learn of the tragedy that has occurred on the beach. The writing is masterful, and the actors do a wonderful job of conveying the couple’s grief and pain. The play is ultimately a reflection on the power of tragedy and the human capacity to confront and overcome it.

Overall, Tragedy at Sunset on the Beach is a must-see for anyone looking for a thought-provoking and emotionally moving play. This play is sure to stay with its audience long after the curtain closes. Highly recommended.


### SequentialChain
Of course, not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain. In this next example, we will experiment with more complex chains that involve multiple inputs, and where there also multiple final outputs.

Of particular importance is how we name the input/output variables. In the above example we didn't have to think about that because we were just passing the output of one chain directly as input to the next, but here we do have worry about that because we have multiple inputs.

In [12]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=0.7)
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)

In [13]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=0.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)


overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nThe play is set in Victorian England and follows the story of a young couple, Mary and John, who were deeply in love and had just gotten engaged. On the night of their engagement, they decided to take a romantic walk along the beach at sunset. Unexpectedly, John is shot by a stranger and killed right in front of Mary. In a state of shock and anguish, Mary is left alone, struggling to comprehend what has just occurred. \n\nThe play follows Mary as she searches for answers to John's death. As Mary's investigation begins, she discovers that John was actually involved in a dark and dangerous plot to overthrow the government. Unbeknownst to Mary, John had been working as a spy in a secret mission to uncover the truth behind a political scandal. \n\nNow, Mary must face the consequences of her beloved's actions and find a way to save the future of England. As the story unfolds, Mary must confront her ow

#### Memory in Sequential Chains
Sometimes you may want to pass along some context to use in each step of the chain or in a later part of the chain, but maintaining and chaining together the input/output variables can quickly get messy.  Using `SimpleMemory` is a convenient way to do manage this and clean up your chains.

For example, using the previous playwright `SequentialChain`, lets say you wanted to include some context about date, time and location of the play, and using the generated synopsis and review, create some social media post text.  You could add these new context variables as `input_variables`, or we can add a `SimpleMemory` to the chain to manage this context:

In [14]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(temperature=0.7)
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(
    input_variables=["synopsis", "review", "time", "location"], template=template
)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(
        memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}
    ),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True,
)

overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': "Experience a heartbreaking love story this Christmas as we bring you 'Tragedy at Sunset on the Beach', set in Victorian England on December 25th at 8pm PST at the Theater in the Park. Follow the story of two young lovers, George and Mary, and their fight against overwhelming odds. Will their love prevail? Find out this Christmas Day! #TragedyAtSunset #LoveStory #Christmas #VictorianEngland"}